---
layout: article
title: fuzz circuits
author: spielhuus
tags: fuzz
cateory: article
logo: verzerrung_files/shin-ei-fy-2-red.jpg
excerpt: comparison and simulation of different fuzz and distortion circuits.
---

this time multiple distortion pedals are simulated. the simultions contains processing and plotting a sine wave, simulate with wave files and analyse the result. the input wave file is a clean guitar. in figure 1 there is a spectrum analysis of the file.

In [ ]:
# Just the normal prequisites for using matplotlib and numpy in a Jupyter notebook
%matplotlib inline
# Use the svg backend, in my opinion it just makes better looking plots
%config InlineBackend.figure_format = 'svg'

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import os
import sys
from pathlib import Path
import IPython.display as ipd
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
import numpy as np
import matplotlib.pyplot as plt
import librosa.display

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

directory_path = Path(os.path.abspath('')).resolve()

In [ ]:
x, sr = librosa.load('151009__khoon__rhythm-1-clean-95-bpm.wav')
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

## overdrive 250

![overdrive 250](verzerrung_files/Overdrive 250.jpg)

The famous DOD 250 Overdrive Preamp Pedal has been reissued and it's a screeching, roaring return! The DOD 250 is ideal for players who like to clean up their tone and just clip the top of the signal. The reissue DOD 250 Overedrive Preamp includes a level and gain control, and that good ol' metal switch.

![overdrive 250 schematic](verzerrung_files/Overdrive_250_schema.svg)

In [ ]:
#load the Overdrive 250 schema
kicad_netlist_path = directory_path.joinpath('Overdrive 250/Overdrive 250.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

class VoltageDivider(SubCircuitFactory):
    __name__ = 'voltage_divider'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4):
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

def overdrivePlot( drive ) :
    overdrive = parser.build_circuit(ground=5)
    overdrive.include(spice_library['TL072'])
    overdrive.include(spice_library['D1N4148'])
    overdrive.V('1', '+15V', overdrive.gnd, 'DC 9')
    overdrive.V('3', 'IN', overdrive.gnd, 'DC 0V AC 0V SIN(0 5V 500)')
    overdrive.subcircuit(VoltageDivider(R=500@u_kΩ, w=drive))
    simulator = overdrive.simulator(temperature=25, nominal_temperature=25)
    return simulator.transient(step_time=1@u_us, start_time=100@u_ms, end_time=104@u_ms)

#plot the results
fig, axis = plt.subplots(figsize=(18, 6))
analysis_shiney = overdrivePlot( 0.1 )    
axis.plot(analysis_shiney['OUT'].abscissa*1000, analysis_shiney['OUT'], c='orange')
analysis_shiney = overdrivePlot( 0.25 )    
axis.plot(analysis_shiney['OUT'].abscissa*1000, analysis_shiney['OUT'], c='LightGreen')
analysis_shiney = overdrivePlot( 0.75 )    
axis.plot(analysis_shiney['OUT'].abscissa*1000, analysis_shiney['OUT'], c='magenta')
analysis_shiney = overdrivePlot( 0.9999 )    
axis.plot(analysis_shiney['OUT'].abscissa*1000, analysis_shiney['OUT'], c='LightBlue') 

axis.grid()
axis.set_xlabel('t [ms]')
axis.set_ylabel('[V]')
axis.set_title('overdrive 250', y=-0.2)
plt.show()


output with guitar sample:

In [ ]:
ipd.Audio(filename='Overdrive 250/build/Overdrive250_151009__khoon__rhythm-1-clean-95-bpm.wav')

In [ ]:
x, sr = librosa.load('Overdrive 250/build/Overdrive250_151009__khoon__rhythm-1-clean-95-bpm.wav')
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

## Shin-ei Companion FY-2

![Shin Ei FY-2](verzerrung_files/shin-ei-fy-2-red.jpg)

The Shin-ei Companion FY-2 is a discontinued fuzz pedal, made by the Japanese Shin-ei effects pedal company from the late 1960s to the early 1970s. The pedal is known for its raw, distinctive, gated fuzz. FY-2 pedals are now rare.

Different versions were made, most using silicon transistors. These had a mid-cutting tone circuit, which lowered the volume. An earlier, rarer, version used germanium transistors. These did not have the tone circuit, and therefore had a louder output. The sound from this version is very rich, fluid and thick.

The pedal has two knobs:

"Volume" (controls overall level)
"Fuzz" (tone)

![Shin Ei FY-2](verzerrung_files/Shin Ei FY-2_schema.svg)

In [ ]:
#load shin ey schema
kicad_netlist_path = directory_path.joinpath('Shin Ei FY-2/Fuzz.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

def shiney( drive )  :
    fuzz = parser.build_circuit(ground=5)
    fuzz.include(spice_library['BC547B'])
    fuzz.V('1', '+15V', fuzz.gnd, 'DC 15')
    fuzz.V('3', 'IN', fuzz.gnd, 'DC 0V AC 0V SIN(0 5V 500)')

    class VoltageDivider(SubCircuitFactory):
        __name__ = 'voltage_divider'
        __nodes__ = ('n1', 'n2', 'n3' )
        def __init__(self, R=100@u_kΩ, w=0.4):
            super().__init__()
            self.R(1, 'n1', 'n2', R * w)
            self.R(2, 'n2', 'n3', R * (1.0-w) )

    fuzz.subcircuit(VoltageDivider(R=100@u_kΩ, w=drive))
    simulator = fuzz.simulator(temperature=25, nominal_temperature=25)
    return simulator.transient(step_time=1@u_us, start_time=100@u_ms, end_time=104@u_ms)

fig, axis = plt.subplots(figsize=(18, 6))
analysis_shiney = shiney( 0.1 )    
axis.plot(analysis_shiney['OUT'].abscissa*1000, analysis_shiney['OUT'], c='orange')
analysis_shiney = shiney( 0.25 )    
axis.plot(analysis_shiney['OUT'].abscissa*1000, analysis_shiney['OUT'], c='LightGreen')
analysis_shiney = shiney( 0.75 )    
axis.plot(analysis_shiney['OUT'].abscissa*1000, analysis_shiney['OUT'], c='magenta')
analysis_shiney = shiney( 0.99 )    
axis.plot(analysis_shiney['OUT'].abscissa*1000, analysis_shiney['OUT'], c='LightBlue')

axis.grid()
axis.set_xlabel('t [ms]')
axis.set_ylabel('[V]')
axis.set_title('Shin Ei FY-2', y=-0.2)

plt.show()

output with guitar sample:

In [ ]:
ipd.Audio(filename='Shin Ei FY-2/build/ShinEiFY-2_151009__khoon__rhythm-1-clean-95-bpm.wav')

In [ ]:
x, sr = librosa.load('Shin Ei FY-2/build/ShinEiFY-2_151009__khoon__rhythm-1-clean-95-bpm.wav')
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

## Way Huge Red LLama

![Way Huge Red LLama](verzerrung_files/Way Huge Red LLama.jpg)

Its space-age circuitry delivers a delicious overdrive that is reminiscent of tube amp distortion and "provides solid tweed-like rhythm tones" Guitar School Magazine (Oct. '96). This pedal has a wide range of dynamic tones from soft and subtle to loud and screaming. Guitar Player Magazine (June '95) says, "[The Red Llama is]...very mean and punchy sounding with just the right amount of fatness a very cool sounding box." Units have volume and drive controls.

More than 25000 guitar effects pedals from 4000 brands: 
http://www.effectsdatabase.com/model/wayhuge/redllama

![Way Huge Red LLama](verzerrung_files/Way Huge Red LLama_schema.svg)

In [ ]:
#load the Huge Red LLama schema
kicad_netlist_path = directory_path.joinpath('Way Huge Red LLama/Way Huge Red LLama.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

def redllama( drive )  :
    redllama_circuit = parser.build_circuit(ground=5)
    redllama_circuit.include(spice_library['CD4069UBhex'])
    redllama_circuit.V('1', '+5V', redllama_circuit.gnd, 'DC 5')
    redllama_circuit.V('3', 'IN', redllama_circuit.gnd, 'DC 0V AC 0V SIN(0 0.1V 500)')

    class VoltageDivider(SubCircuitFactory):
        __name__ = 'voltage_divider'
        __nodes__ = ('n1', 'n2', 'n3' )
        def __init__(self, R=100@u_kΩ, w=0.4):
            super().__init__()
            self.R(1, 'n1', 'n2', R * w)
            self.R(2, 'n2', 'n3', R * (1.0-w) )

    redllama_circuit.subcircuit(VoltageDivider(R=100@u_kΩ, w=drive))
    simulator = redllama_circuit.simulator(temperature=25, nominal_temperature=25)
    return simulator.transient(step_time=1@u_us, start_time=100@u_ms, end_time=104@u_ms)

fig, axis = plt.subplots(figsize=(18, 6))
analysis_redllama = redllama( 0.1 )    
axis.plot(analysis_redllama['OUT'].abscissa*1000, analysis_redllama['OUT'], c='orange')
analysis_redllama = redllama( 0.25 )    
axis.plot(analysis_redllama['OUT'].abscissa*1000, analysis_redllama['OUT'], c='LightGreen')
analysis_redllama = redllama( 0.75 )    
axis.plot(analysis_redllama['OUT'].abscissa*1000, analysis_redllama['OUT'], c='magenta')
analysis_redllama = redllama( 0.99 )    
axis.plot(analysis_redllama['OUT'].abscissa*1000, analysis_redllama['OUT'], c='LightBlue')

axis.grid()
axis.set_xlabel('t [ms]')
axis.set_ylabel('[V]')
axis.set_title('Way Huge Red LLama', y=-0.2)

plt.show()

output with guitar sample

In [ ]:
ipd.Audio(filename='Way Huge Red LLama/build/WayHugeRedLLama_151009__khoon__rhythm-1-clean-95-bpm.wav')

In [ ]:
x, sr = librosa.load('Way Huge Red LLama/build/WayHugeRedLLama_151009__khoon__rhythm-1-clean-95-bpm.wav')
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

## UBE Screamer


![UBE Screamer](verzerrung_files/UBE Screamer_schema.svg)

In [ ]:
#load the UBE Screamer
kicad_netlist_path = directory_path.joinpath('UBE Screamer/UBE Screamer.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

def ube( drive )  :
    ube_circuit = parser.build_circuit(ground=5)
    ube_circuit.include(spice_library['CD4069UBhex'])
    ube_circuit.include(spice_library['D1N4148'])
    ube_circuit.V('1', '+5V', ube_circuit.gnd, 'DC 5')
    ube_circuit.V('3', 'IN', ube_circuit.gnd, 'DC 0V AC 0V SIN(0 0.5V 500)')

    class VoltageDivider1(SubCircuitFactory):
        __name__ = 'voltage_divider_1'
        __nodes__ = ('n1', 'n2', 'n3' )
        def __init__(self, R=100@u_kΩ, w=0.4):
            super().__init__()
            self.R(1, 'n1', 'n2', R * w)
            self.R(2, 'n2', 'n3', R * (1.0-w) )

    class VoltageDivider2(SubCircuitFactory):
        __name__ = 'voltage_divider_2'
        __nodes__ = ('n1', 'n2', 'n3' )
        def __init__(self, R=100@u_kΩ, w=0.4):
            super().__init__()
            self.R(1, 'n1', 'n2', R * w)
            self.R(2, 'n2', 'n3', R * (1.0-w) )

    ube_circuit.subcircuit(VoltageDivider1(R=500@u_kΩ, w=drive))
    ube_circuit.subcircuit(VoltageDivider2(R=500@u_kΩ, w=0.5))
    simulator = ube_circuit.simulator(temperature=25, nominal_temperature=25)
    return simulator.transient(step_time=1@u_us, start_time=100@u_ms, end_time=110@u_ms)

fig, axis = plt.subplots(figsize=(18, 6))
analysis_ube= ube( 0.1 )    
axis.plot(analysis_ube['OUT'].abscissa*1000, analysis_ube['OUT'], c='orange') 
analysis_ube = ube( 0.25 )    
axis.plot(analysis_ube['OUT'].abscissa*1000, analysis_ube['OUT'], c='LightGreen')
analysis_ube = ube( 0.75 )    
axis.plot(analysis_ube['OUT'].abscissa*1000, analysis_ube['OUT'], c='magenta')
analysis_ube = ube( 0.99 )    
axis.plot(analysis_ube['OUT'].abscissa*1000, analysis_ube['OUT'], c='LightBlue') 

axis.grid()
axis.set_xlabel('t [ms]')
axis.set_ylabel('[V]')
axis.set_title('UBE Screamer', y=-0.2)

plt.show()

output with guitar sample

In [ ]:
ipd.Audio(filename='UBE Screamer/build/UBEScreamer_151009__khoon__rhythm-1-clean-95-bpm.wav')

In [ ]:
x, sr = librosa.load('UBE Screamer/build/UBEScreamer_151009__khoon__rhythm-1-clean-95-bpm.wav')
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

[1]: http://beavisaudio.com/schematics/DOD-250-Overdrive-Schematic.htm
[2]: https://www.diystompboxes.com/smfforum/index.php?topic=118397.0
[3]: https://www.diystompboxes.com/smfforum/index.php?topic=99779.0
[4]: http://www.runoffgroove.com/ubescreamer.html
[5]: http://beavisaudio.com/schematics/Way-Huge-Red-Llama-Schematic.htm
[6]: https://static1.squarespace.com/static/5e6e99e8bd2f8a6de8454feb/t/5e77db16e8ff44374c24b124/1584913176675/MBD+BOM+and+build.pdf
